In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai import *

In [4]:
2 + 2

4

# Rossmann

## Data preparation / Feature engineering

In addition to the provided data, we will be using external datasets put together by participants in the Kaggle competition. You can download all of them [here](http://files.fast.ai/part2/lesson14/rossmann.tgz). Then you shold untar them in the dirctory to which `PATH` is pointing below.

For completeness, the implementation used to put them together is included below.

In [5]:
PATH=Path('data/rossmann/')
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(PATH/f'{fname}.csv', low_memory=False) for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train),len(test)

(1017209, 41088)

In [6]:
googletrend.head(3)

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91


We turn state Holidays to booleans, to make them more convenient for modeling. We can do calculations on pandas fields using notation very similar (often identical) to numpy.

In [7]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [8]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

Join weather/state names.

In [9]:
weather = join_df(weather, state_names, "file", "StateName")

In pandas you can add new columns to a dataframe by simply defining it. We'll do this for googletrends by extracting dates and state names from the given data and adding those columns.

We're also going to replace all instances of state name 'NI' to match the usage in the rest of the data: 'HB,NI'. This is a good opportunity to highlight pandas indexing. We can use `.loc[rows, cols]` to select a list of rows and a list of columns from the dataframe. In this case, we're selecting rows w/ statename 'NI' by using a boolean list `googletrend.State=='NI'` and selecting "State".

In [10]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [11]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [12]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

In [15]:
googletrend['file'].head(3)

0    Rossmann_DE_SN
1    Rossmann_DE_SN
2    Rossmann_DE_SN
Name: file, dtype: object

The Google trends data has a special category for the whole of the Germany - we'll pull that out so we can use it explicitly.

In [16]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [18]:
trend_de.count()

file                148
week                148
trend               148
Date                148
State                 0
Year                148
Month               148
Week                148
Day                 148
Dayofweek           148
Dayofyear           148
Is_month_end        148
Is_month_start      148
Is_quarter_end      148
Is_quarter_start    148
Is_year_end         148
Is_year_start       148
Elapsed             148
dtype: int64

Now we can outer join all of our data into a single dataframe. Recall that in outer joins everytime a value in the joining field on the left table does not have a corresponding value on the right table, the corresponding row in the new table has Null values for all right table fields. One way to check that all records are consistent and complete is to check for Null values post-join, as we do here.

*Aside*: Why note just do an inner join?
If you are assuming that all records are complete and match on the field you desire, an inner join will do the same thing as an outer join. However, in the event you are wrong or a mistake is made, an outer join followed by a null-check will catch it. (Comparing before/after # of rows for inner join is equivalent, but requires keeping track of before/after row #'s. Outer join is easier.)

In [19]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [20]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [21]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [22]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [23]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [24]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

Next we'll fill in missing values to avoid complications with `NA`'s. `NA` (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary *signal value* that doesn't otherwise appear in the data.

In [25]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

Next we'll extract features "CompetitionOpenSince" and "CompetitionDaysOpen". Note the use of `apply()` in mapping a function across dataframe values.

In [26]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

We'll replace some erroneous / outlying data.

In [27]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

We add "CompetitionMonthsOpen" field, limiting the maximum to 2 years to limit number of unique categories.

In [28]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5])

Same process for Promo dates. You may need to install the `isoweek` package first.

In [31]:
# If needed, uncomment:
# ! sudo pip install isoweek

In [42]:
# DON'T DO THIS

# import sys
# sys.path.append('./.local/lib/python3.6/site-packages/isoweek.py')
# sys.path

['',
 '/opt/anaconda3/lib/python36.zip',
 '/opt/anaconda3/lib/python3.6',
 '/opt/anaconda3/lib/python3.6/lib-dynload',
 '/opt/anaconda3/lib/python3.6/site-packages',
 '/opt/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/jupyter/.ipython',
 './.local/lib/python3.6/site-packages',
 './.local/lib/python3.6/site-packages',
 './.local/lib/python3.6/site-packages/isoweek.py']

In [44]:
import isoweek

ModuleNotFoundError: No module named 'isoweek'

In [45]:
from datetime import date, datetime, timedelta
from collections import namedtuple

__version__ = (1, 3, 3)


import sys
if sys.version >= '3':
    # compatiblity tweaks
    basestring = str
    long = int

class Week(namedtuple('Week', ('year', 'week'))):
    """A Week represents a period of 7 days starting with a Monday.
    Weeks are identified by a year and week number within the year.
    This corresponds to the read-only attributes 'year' and 'week'.

    Week 1 of a year is defined to be the first week with 4 or more days in
    January.  The preceeding week is either week 52 or 53 of the
    preceeding year.

    Week objects are tuples, and thus immutable, with an interface
    similar to the standard datetime.date class.
    """
    __slots__ = ()

    def __new__(cls, year, week):
        """Initialize a Week tuple with the given year and week number.

        The week number does not have to be within range.  The numbers
        will be normalized if not.  The year must be within the range
        1 to 9999.
        """
        if week < 1 or week > 52:
            return cls(year, 1) + (week - 1)
        if year < 1 or year > 9999:
            raise ValueError("year is out of range")
        return super(Week, cls).__new__(cls, year, week)

    @classmethod
    def thisweek(cls):
        """Return the current week (local time)."""
        return cls(*(date.today().isocalendar()[:2]))

    @classmethod
    def fromordinal(cls, ordinal):
        """Return the week corresponding to the proleptic Gregorian ordinal,
        where January 1 of year 1 starts the week with ordinal 1.
        """
        if ordinal < 1:
            raise ValueError("ordinal must be >= 1")
        return super(Week, cls).__new__(cls, *(date.fromordinal((ordinal-1) * 7 + 1).isocalendar()[:2]))

    @classmethod
    def fromstring(cls, isostring):
        """Return a week initialized from an ISO formatted string like "2011W08" or "2011-W08"."""
        if isinstance(isostring, basestring) and len(isostring) == 7 and isostring[4] == 'W':
           return cls(int(isostring[0:4]), int(isostring[5:7]))
        elif isinstance(isostring, basestring) and len(isostring) == 8 and isostring[4:6] == '-W':
           return cls(int(isostring[0:4]), int(isostring[6:8]))
        else:
            raise ValueError("Week.tostring argument must be on the form <yyyy>W<ww>; got %r" % (isostring,))

    @classmethod
    def withdate(cls, date):
        """Return the week that contains the given datetime.date"""
        return cls(*(date.isocalendar()[:2]))

    @classmethod
    def weeks_of_year(cls, year):
        """Return an iterator over the weeks of the given year.
        Years have either 52 or 53 weeks."""
        w = cls(year, 1)
        while w.year == year:
            yield w
            w += 1

    @classmethod
    def last_week_of_year(cls, year):
        """Return the last week of the given year.
        This week with either have week-number 52 or 53.

        This will be the same as Week(year+1, 0), but will even work for
        year 9999 where this expression would overflow.

        The first week of a given year is simply Week(year, 1), so there
        is no dedicated classmethod for that.
        """
        if year == cls.max.year:
            return cls.max
        return cls(year+1, 0)

    def day(self, num):
        """Return the given day of week as a date object.  Day 0 is the Monday."""
        d = date(self.year, 1, 4)  # The Jan 4th must be in week 1 according to ISO
        return d + timedelta(weeks=self.week-1, days=-d.weekday() + num)

    def monday(self):
        """Return the first day of the week as a date object"""
        return self.day(0)

    def tuesday(self):
        """Return the second day the week as a date object"""
        return self.day(1)

    def wednesday(self):
        """Return the third day the week as a date object"""
        return self.day(2)

    def thursday(self):
        """Return the fourth day the week as a date object"""
        return self.day(3)

    def friday(self):
        """Return the fifth day the week as a date object"""
        return self.day(4)

    def saturday(self):
        """Return the sixth day the week as a date object"""
        return self.day(5)

    def sunday(self):
        """Return the last day the week as a date object"""
        return self.day(6)

    def days(self):
        """Return the 7 days of the week as a list (of datetime.date objects)"""
        monday = self.day(0)
        return [monday + timedelta(days=i) for i in range(7)]

    def contains(self, day):
        """Check if the given datetime.date falls within the week"""
        return self.day(0) <= day < self.day(7)

    def toordinal(self):
        """Return the proleptic Gregorian ordinal the week, where January 1 of year 1 starts the first week."""
        return self.monday().toordinal() // 7 + 1

    def replace(self, year=None, week=None):
        """Return a Week with either the year or week attribute value replaced"""
        return self.__class__(self.year if year is None else year,
                              self.week if week is None else week)

    def year_week(self):
        """Return a regular tuple containing the (year, week)"""
        return self.year, self.week

    def __str__(self):
        """Return a ISO formatted week string like "2011W08". """
        return '%04dW%02d' % self

    isoformat = __str__  # compatibility with datetime.date

    def __repr__(self):
        """Return a string like "isoweek.Week(2011, 35)"."""
        return __name__ + '.' + self.__class__.__name__ + '(%d, %d)' % self

    def __add__(self, other):
        """Adding integers to a Week gives the week that many number of weeks into the future.
        Adding with datetime.timedelta is also supported.
        """
        if isinstance(other, timedelta):
            other = other.days // 7
        return self.__class__.fromordinal(self.toordinal() + other)

    def __sub__(self, other):
        """Subtracting two weeks give the number of weeks between them as an integer.
        Subtracting an integer gives another Week in the past."""
        if isinstance(other, (int, long, timedelta)):
            return self.__add__(-other)
        return self.toordinal() - other.toordinal()

Week.min = Week(1,1)
Week.max = Week(9999,52)
Week.resolution = timedelta(weeks=1)

In [48]:
# from isoweek import Week
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [49]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [50]:
joined.to_pickle(PATH/'joined')
joined_test.to_pickle(PATH/'joined_test')

## Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [51]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

We'll be applying this to a subset of columns:

In [52]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [53]:
#df = train[columns]
df = train[columns].append(test[columns])

Let's walk through an example.

Say we're looking at School Holiday. We'll first sort by Store, then Date, and then call `add_elapsed('SchoolHoliday', 'After')`:
This will apply to each row with School Holiday:
* A applied to every row of the dataframe in order of store and date
* Will add to the dataframe the days since seeing a School Holiday
* If we sort in the other direction, this will count the days until another holiday.

In [54]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We'll do this for two more fields.

In [55]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [56]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

We're going to set the active index to Date.

In [57]:
df = df.set_index("Date")

Then set null values from elapsed field calculations to 0.

In [58]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [59]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

Next we'll demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (`sort_index()`) and counting the number of events of interest (`sum()`) defined in `columns` in the following week (`rolling()`), grouped by Store (`groupby()`). We do the same in the opposite direction.

In [60]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [61]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

Next we want to drop the Store indices grouped together in the window function.

Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [62]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [63]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [64]:
df.reset_index(inplace=True)

Now we'll merge these values onto the df.

In [65]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [66]:
df.drop(columns,1,inplace=True)

In [67]:
df.head()

,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,13,0,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,1,12,0,104,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1,11,0,103,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,1,10,0,102,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,1,9,0,101,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0


It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [68]:
df.to_pickle(PATH/'df')

In [69]:
df["Date"] = pd.to_datetime(df.Date)

In [70]:
df.columns

Index(['Date', 'Store', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday', 'BeforeStateHoliday', 'AfterPromo', 'BeforePromo',
       'SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw',
       'StateHoliday_fw', 'Promo_fw'],
      dtype='object')

In [71]:
joined = pd.read_pickle(PATH/'joined')
joined_test = pd.read_pickle(PATH/f'joined_test')

In [72]:
joined = join_df(joined, df, ['Store', 'Date'])

In [73]:
joined_test = join_df(joined_test, df, ['Store', 'Date'])

The authors also removed all instances where the store had zero sale / was closed. We speculate that this may have cost them a higher standing in the competition. One reason this may be the case is that a little exploratory data analysis reveals that there are often periods where stores are closed, typically for refurbishment. Before and after these periods, there are naturally spikes in sales that one might expect. By ommitting this data from their training, the authors gave up the ability to leverage information about these periods to predict this otherwise volatile behavior.

In [74]:
joined = joined[joined.Sales!=0]

We'll back this up as well.

In [75]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [76]:
joined.to_pickle(PATH/'train_clean')
joined_test.to_pickle(PATH/'test_clean')